In [15]:
import requests
from io import BytesIO, StringIO
from zipfile import ZipFile
import pandas as pd

from lib import get_state_codes

# get these from your login
SESSION_COOKIE = 's%3AMaQpNUwZwsmig9fE0XDZzk9K0xJwcRP4.vYJT06RO4nVDV9gotB4x96YdaXXLsRrBW5dY5nnp1Vs'
_csrf = 'pyGlYwW2-MaI7CkYx6eL6fLQnSiYDf1lG5R4'

def _fetch_data(state_code: str = "AL") -> BytesIO:
    print(state_code)
    
    cookies = {
        'sess': SESSION_COOKIE,
    }

    data = {
        'state-type': 'zipcode',
        '_csrf': _csrf,
        'scale-group': 'state',
        'state-name': state_code,
        'version-group': '19',
    }

    response = requests.post('https://www.neighborhoodatlas.medicine.wisc.edu/adi-download', cookies=cookies, data=data)
    print(response.status_code)
    return BytesIO(response.content)

def build_adi_table(state_code: str) -> None:
    zf = ZipFile(_fetch_data(state_code))
    content = StringIO(zf.read(zf.namelist()[0]).decode())
    df = pd.read_csv(content, dtype="str")
    df.to_parquet(f"data/state_tables/{state_code}.parquet")

for state in get_state_codes():
    build_adi_table(state)


200
